In [ ]:
!pip install pygad

In [ ]:
import tensorflow.keras
import pygad.kerasga
import numpy
import pygad
import pandas

Defining Fitness Function

In [ ]:
def fitness_func(solution, sol_idx):
    global data_inputs, data_outputs, keras_ga, model
    model_weights_matrix = pygad.kerasga.model_weights_as_matrix(model=model,
                                                                 weights_vector=solution)
    model.set_weights(weights=model_weights_matrix)
    predictions = model.predict(data_inputs)
    mape = tensorflow.keras.losses.MeanAbsolutePercentageError()
    abs_error = mape(data_outputs, predictions).numpy() + 0.00000001
    solution_fitness = 1.0 / abs_error
    return solution_fitness

Callback generation

In [ ]:
def callback_generation(ga_instance):
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))

Setting ANN layers

In [ ]:
input_layer  = tensorflow.keras.layers.Input(11)
dense_layer1 = tensorflow.keras.layers.Dense(30, activation="relu")(input_layer)
dense_layer2 = tensorflow.keras.layers.Dense(30, activation="relu")(input_layer)
output_layer = tensorflow.keras.layers.Dense(1, activation="linear")(dense_layer1)

model = tensorflow.keras.Model(inputs=input_layer, outputs=output_layer)
weights_vector = pygad.kerasga.model_weights_as_vector(model=model)
keras_ga = pygad.kerasga.KerasGA(model=model,
                                 num_solutions=30) 

Data input and output

In [ ]:
data = numpy.array(pandas.read_csv("npc 18 test for colab 5.csv"))
# Preparing the NumPy array of the inputs.
data_inputs = numpy.asarray(data[:,:11], dtype=numpy.float32)

# Preparing the NumPy array of the outputs.
data_outputs = numpy.asarray(data[:,11], dtype=numpy.float32)

In [ ]:
num_generations = 2000
num_parents_mating = 5
initial_population = keras_ga.population_weights
print(len(initial_population))
ga_instance = pygad.GA(num_generations=num_generations, 
                       num_parents_mating=num_parents_mating, 
                       initial_population=initial_population,
                       fitness_func=fitness_func,
                       on_generation=callback_generation
                        )
ga_instance.run()

In [ ]:
# After the generations complete, some plots are showed that summarize how the outputs/fitness values evolve over generations.
ga_instance.plot_result(title="PyGAD & Keras - Iteration vs. Fitness", linewidth=4)

# Returning the details of the best solution.
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

# Fetch the parameters of the best solution.
best_solution_weights = pygad.kerasga.model_weights_as_matrix(model=model,
                                                              weights_vector=solution)
model.set_weights(best_solution_weights)
predictions = model.predict(data_inputs)
print("Predictions : \n", predictions)

mape = tensorflow.keras.losses.MeanAbsolutePercentageError()
abs_error = mape(data_outputs, predictions).numpy()
print("mape: ", abs_error)

In [ ]:
#import pygad.nn
#import pygad.gann

#GANN_instance=pygad.gann.GANN(num_solutions=5,
 #                               num_neurons_input=2,
  #                              num_neurons_hidden_layers=[2],
   #                             num_neurons_output=1,
    #                            hidden_activations=["relu"],
     #                           output_activation="softmax")
#predictions = pygad.nn.predict(last_layer=GANN_instance.population_networks[solution_idx],
 #                              data_inputs=data_inputs,
  #                            problem_type="regression")
#print("Predictions of the trained network : {predictions}".format(predictions=predictions))